## **Basics Scraping with Scrapy**

### **Setup Environment and Packages**

Untuk memulai menggunakan Scrapy sangat disarankan untuk kita menggunakan virtual environment. Virtual environment merupakan sebuah ruang isolasi yang memungkinkan kita untuk menggunakannya secara virtual (terpisah dari penyimpanan global) untuk menyimpan segala kebutuhan packages yang akan digunakan pada suatu project. Hal ini penting karena akan memudahkan kita dalam memenuhi kebutuhan dari project satu dengan yang lainnya.

Pertama-tama kita perlu setup dan install packages, dengan cara:

**1. buka cmd/terminal/bash**
**2. buat virtual environment**
 ```bash
    python -m venv env # Windows # env merupakan nama virtual environement bisa berikan nama sesuai kebutuhan
    python3 -m venv env # MacOS
```
**3. aktifkan virtual environment**
 ```bash
    env\Srcipts\activate # Windows
    source env\bin\activate # MacOS
```
**4. install requirements yang sudah disediakan atau install mandiri dengan menulis nama packages**
```bash
    pip install -r requirements.txt
```
**5. cek list packages yang terinstall**
```bash
    pip list
```
**6. deactivate jika sudah selesai menggunakan virtual environment**
```bash
    deactivate
```

**Tambahan**: Ketika anda menggunakan virtual enviroment, cek apakah sudah terdapat di jupyter kernel anda. Jika belum terdaftar di kernel, anda bisa menginstall ipykernel dan daftarkan env ke kernel dengan membuat name dan display name agar terlihat di vscode. Berikut perintah yang digunakan:

```bash
pip install ipykernel
python -m ipykernel install --user --name=myenv --display-name "Python (scrapy-env)"
```

### **Setup Scrapy & Spiders**

Pastikan env(virtual environment yang sudah dibuat) aktif dan berada di root directory project anda.

**1. buat folder untuk project scrapy anda**
```bash
scrapy startproject scrapy_project
```
scrapy akan otomatis membuat folder sesuai nama yang dibuat dan telah berisi segala kebutuhan file python lainnya.

```
root_project/
│
├── scrapy_project/               <-- folder proyek scrapy
│   ├── scrapy_project/
│   │   ├── __init__.py
│   │   ├── items.py
│   │   ├── middlewares.py
│   │   ├── pipelines.py
│   │   ├── settings.py
│   │   └── spiders/         <-- tempat kamu bikin file spider
│   │       └── __init__.py
│   └── scrapy.cfg           <-- konfigurasi utama scrapy
│
├── notebooks/               <-- folder kumpulan jupyter notebooks
```

**2. buat spiders untuk memulai melakukan project scraping**
```bash
scrapy genspiders exapmple example.com
```
perintah diatas akan membuat nama spider **example** dan membuat template scrapy sesuai url yang menjadi target scraping yaitu **example.com**.   

```
scrapy_project/               <-- folder proyek scrapy
    ├── scrapy_project/
    │   ├── __pycahche__/
    │   ├── __init__.py
    │   ├── items.py
    │   ├── middlewares.py
    │   ├── pipelines.py
    │   ├── settings.py
    │   └── spiders/         <-- tempat kamu bikin file spider
    │       ├──__pycahche__/
    │       ├──__init__.py
    │       └── example.py
    └── scrapy.cfg
```  

**Template isi file exapmle.py**

```Python
import scrapy


class ExampleSpider(scrapy.Spider):
    name = "example"
    allowed_domains = ["www.example.info"]
    start_urls = ["https://www.example.info/world-population/population-by-country"]

    def parse(self, response):
        yield
```

## **Scrapy Bascics for Scraping**

![worldometers population](../media/worldometers_scrapy.png)

Untuk memulai buka terminal/cmd anda dan pastikan env sudah aktif dan directory berada pada folder project scrapy anda.

Jika semua sudah siap, anda bisa memulai dengan mengetahui beberapa dasar awal yang sering digunakan.

**1. `scrapy shell`**
Perintah ini akan membuat kita memasuki tools interaktif dari Scrapy untuk melakukan penujian dan debug sebelum melakukan scraping. Kita kan memulai dengan melakukan request terhadap target web yang ingin discraping.

Dalam kasus ini saya menggunakan website worldometers.info. Perintah yang dilakukan sebagai berikut:
```bash
scrapy shell
In [1]: r = scrapy.Request('https://www.worldometers.info/world-population/population-by-country/')
In [2]: fetch(r)
```
Perintah Request adalah untuk mendapatkan request dari website yang kita tuju dan fetch untuk mendapatkan response untuk hasilnya.

**2. lakukan perintah untuk mendapatkan hasil dari website**
```bash
In [3]: response.body

In [4]: response.xpath('//h1')
Out[4]: [<Selector query='//h1' data='<h1 id="countries-in-the-world-by-pop...'>]

In [5]: response.xpath('//h1/text()')
Out[5]: [<Selector query='//h1/text()' data='Countries in the world by population ...'>]

In [6]: response.xpath('//h1/text()').get()
Out[6]: 'Countries in the world by population (2025)'

In [7]: response.xpath('//td/a/text()').get()
Out[7]: 'India'

In [8]: response.xpath('//td/a/text()').getall()
Out[8]:
['India',
 'China',
 'United States',
 'Indonesia',
 'Pakistan',
 'Nigeria',
 ...]
```
`response.body` = untuk mengambil hasil dari seluruh body struktur HTML website

`response.xpath` = untuk mengambil hasil dari xpath yang diuju

`/text()` = cara untuk mengambil teks yang terdapat pada suatu tag/path yang ada di stuktur HTML

`.get/.getall` = untuk mengamil hail/untuk mengamil semua hasil yang memiliki xpath yang sama


Jika sudah dilakukan testing di Scrapy shell kita bisa mulai memasukkan kode kedalam project kita, dan running.

```Python
import scrapy


class WorldometersSpider(scrapy.Spider):
    name = "worldometers"
    allowed_domains = ["www.worldometers.info"]
    start_urls = ["https://www.worldometers.info/world-population/population-by-country"]

    def parse(self, response):
        title = response.xpath('//h1/text()').get()
        countries = response.xpath('//td/a').getall()

            
        yield {
                'title' : title,
                'country': countries,
        }
```

Untuk running anda bisa gunakan perintah pada scrapy:
```bash
scrapy crawl worldometers
```

### **Scraping Data from Multiple Links** 

Sebelum mengerti mekanisme untuk mengambil link menggunakan Scrapy, kita perlu tahu apa itu **absolute** dan **relative** link.

### **Absolute & Relative Link**

Relative link merupakan tautan link yang tidak menyertakan alamat lengkap URL teramasuk nama domain dan protokolnya. Realative link hanya menunjukkan jalur menuju sumber halaman yang dituju. Berbeda dengan absolute link yang merupakan link lengkap dengan domain dan protokolnya. Contoh link untuk mamahami absolute dan relative link.

**Absolute link** : "https//:exapmle.com"

**Relative link** : "/example-info/example-1.html"

### **Get Multiple Link**

![worldometers-table](../media/worldometers_table_scrapy.png)

Jika kita lihat dari href dapat kita lihat relative link merujuk kepada halaman khusus pada negara India.

![worldometers-relative_link](../media/worldometers_scrapy_relative_link.png)


Dalam Scrapy untuk bisa melakukan request pada relative link, ada beberapa cara.

Pertama-tama kita perlu mengambil attribut href dengan cara

Cara manual, ktia bisa melakukan inisiasi absolute link + relative link, lalu melakukan request.

Tetapi di Scrapy kita bisa mudah melakukannya tanpa harus melakukannya secara manual, kita hanya perlu menggunakan fungsi `follow(url)`

Contoh kodenya akan seperti ini:
```Python
class WorldometersSpider(scrapy.Spider):
    name = "worldometers"
    allowed_domains = ["www.worldometers.info"]
    start_urls = ["https://www.worldometers.info/world-population/population-by-country"]

    def parse(self, response):
        # title = response.xpath('//h1/text()').get()
        countries = response.xpath('//td/a')

        for country in countries:
            country_name = country.xpath('.//text()').get()
            link = country.xpath('.//@href').get()
            


            # untuk menngambil link lengkap ada dua cara

            # 1. Menggunakan response.urljoin
            # absoulute_url = f'https://www.worldometers.info{link}'
            # yield response.urljoin(url=abosolute_url)

            # 2. Menggunakan response.follow
            # yield response.follow(url=link) # cukup gunakan link relatif

            # untuk mengembalikan data dalam bentuk dictionary
            yield {
                "countries": country_name,
                "links": link,
            }
```

### **Request to Each Link and Crawl The Data**

```Python
class WorldometersSpider(scrapy.Spider):
    name = "worldometers"
    allowed_domains = ["www.worldometers.info"]
    start_urls = ["https://www.worldometers.info/world-population/population-by-country"]

    def parse(self, response):
        # title = response.xpath('//h1/text()').get()
        countries = response.xpath('//td/a')

        for country in countries:
            country_name = country.xpath('.//text()').get()
            link = country.xpath('.//@href').get()

            yield response.follow(url=link, callback=self.parse_country, meta={"country": country_name})

    def parse_country(self, response):
        # untuk mengambil data dari halaman negara
        rows = response.xpath('(//table[contains(@class, "table")])[1]//tbody/tr')

        country = response.request.meta['country']
        for row in rows:
            year = row.xpath('.//td[1]/text()').get()
            population = row.xpath('.//td[2]/text()').get()

            yield {
                "country": country,
                "year": year,
                "population": population,
            }
```

Kode di atas melakuan crawling pada xpath awal sepeti ini:
```xpath
(//table[@class="datatable w-full border border-zinc-200 datatable-table"])[1]
```

Untuk mengambil data dari setiap baris tabel yang terdapat pada tag `<tr>` dan `<td>` kita perlu masukkan xpath sesuai dengan data yang ingin diambil. Namun, kita perlu membuat fungsi tersendiri untuk mengambil data dari relative link yang sudah didapat. 

Ketika sudah membuat fungsi untuk crawling data pada relative link, 
```Python
yield response.follow(url=link, callback=self.parse_country, meta={"country": country_name})
``` 
baris kode ini akan melakukan request dan mendapatkan respon dari link yang didapatkan dan callback pada fungsi yang sudah dibuat untuk mengambil data, meta berfungsi sebagai data tambahan yang sudah kita ambil sebelumnya untuk menghubungkan data yang telah diambil sebelumnya.

Untuk melakukan running kita bisa memulai dengan crawling
```bash
scrapy crawls worldometers
```
Perhatikan jika kode response (200) maka data berhasil di crawling. Sekarang untuk bisa melakukan exporting data, kita hanya perlu melakukan perintah pada env kita dengan:
```bash
scrapy crawls -o data_export_exampple.csv
```
atau 
```bash
scrapy crawls -o "path_folder/data_export_example.json"
```